In [166]:
import gensim
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join


In [167]:
docLabels = []
docLabels = [f for f in listdir("data") if 
 f.endswith(".txt")]

In [168]:
print (docLabels)

['1.txt', '10.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt', '8.txt', '9.txt']


In [169]:
data = []
for doc in docLabels:
    data.append(open("data/" + doc).read())

In [170]:
print (data[0][0:10])

Cricket is


In [171]:
tokenizer = RegexpTokenizer(r'\w+')
stopword_set = set(stopwords.words('english'))
#This function does all cleaning of data using two objects above
def nlp_clean(data):
    new_data = []
    for d in data:
        new_str = d.lower()
        dlist = tokenizer.tokenize(new_str)
        dlist = list(set(dlist).difference(stopword_set))
        new_data.append(dlist)
    return new_data

In [172]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.LabeledSentence(doc,[self.labels_list[idx]])

In [173]:
data = nlp_clean(data)

In [174]:
print (data[0])

['result', 'two', 'innings', 'centre', 'gained', 'scores', 'depending', 'wicket', 'played', 'match', 'team', 'opponents', 'ball', 'next', 'except', 'long', 'phase', 'yard', 'including', 'teams', 'apiece', 'runs', 'roles', 'play', 'bats', 'field', 'rectangular', 'eleven', 'type', 'called', 'target', 'whilst', 'set', 'winning', 'matches', 'wooden', 'stumps', 'attempting', 'end', 'swap', 'players', 'bails', 'extras', 'first', 'ends', 'one', 'cricket', '22', 'topped', 'possible', 'three', 'many', 'draw', 'game', 'pitch', 'score', 'bat']


In [175]:
it = LabeledLineSentence(data, docLabels)

In [176]:
model = gensim.models.Doc2Vec(size=300, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(it)
#training of model
for epoch in range(100):
    #print ('iteration '+str(epoch+1))
    model.train(it, total_examples=model.corpus_count,epochs=model.iter)
    model.alpha -= 0.002
    model.min_alpha = model.alpha
#saving the created model
#model.save(‘doc2vec.model’)
#print (“model saved”)

In [177]:
##loading the model
#d2v_model = gensim.models.doc2vec.Doc2Vec.load(‘doc2vec.model’)

In [178]:
d2v_model = model
docvec = d2v_model.docvecs[0]
#print (docvec)

In [179]:
docvec = d2v_model.docvecs['1.txt'] #if string tag used in training
#print (docvec)  #same but refeences with label

In [180]:
sims = d2v_model.docvecs.most_similar('1.txt')
print (sims)

[('3.txt', 0.9999394416809082), ('10.txt', 0.9998970031738281), ('2.txt', 0.9998953938484192), ('7.txt', 0.9997853636741638), ('6.txt', 0.9997463226318359), ('4.txt', 0.9997228980064392), ('8.txt', 0.9996271729469299), ('5.txt', 0.9995402097702026), ('9.txt', 0.9993734359741211)]


In [181]:
#to get vector of document that are not present in corpus 
docvec = d2v_model.infer_vector(["google","is","a","search", "engine"])
#print (docvec)

In [182]:
d2v_model.docvecs.most_similar([docvec])

[('1.txt', -0.9998027086257935),
 ('9.txt', -0.9998462200164795),
 ('5.txt', -0.9999169111251831),
 ('8.txt', -0.9999468922615051),
 ('3.txt', -0.9999501705169678),
 ('10.txt', -0.9999712109565735),
 ('4.txt', -0.9999712109565735),
 ('2.txt', -0.9999716281890869),
 ('6.txt', -0.9999752640724182),
 ('7.txt', -0.9999803304672241)]